In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')

In [2]:
%load_ext autoreload
%autoreload 2
#import CCPA_lib as cp
import sorf_lib as sf

In [3]:
sns.set_context('poster')


![](http://oregonstate.edu/instruct/bb450/fall14/stryer7/2/table_02_02.jpg)

https://www.uniprot.org/uniprot/Q7V735

http://tigrfams.jcvi.org/cgi-bin/HmmReportPage.cgi?acc=TIGR03798

https://www.ebi.ac.uk/training/online/course/interpro-functional-and-structural-analysis-protei/sequence-searching/searching-interpro-batc

http://www.ebi.ac.uk/interpro/sequencesearch/iprscan5-S20190707-131508-0462-76111813-p1m

https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/iprscan5-S20190707-131508-0462-76111813-p1m/json

In [4]:
genome_df = sf.load_genome('MIT9313')

In [5]:
genome_df.head()

,contig_id,gene_id,feature_id,type,location,start,stop,strand,function,aliases,figfam,evidence_codes,nucleotide_sequence,aa_sequence,genome,left,right
0,MIT9313,PMIT9313_0001,fig|1218.91.peg.1,peg,MIT9313_174_1340,174,1340,+,DNA polymerase III beta subunit (EC 2.7.7.7),NaN,FIG00066425,isu;DNA-replication isu;DNA_replication_cluster_1,atgaaactggtttgttcacaggctgaactcaatacagcactacagc...,MKLVCSQAELNTALQLVSRAVASRPTHPVLANVLLTADAGTDRLSL...,MIT9313,174,1340
1,MIT9313,PMIT9313_0002,fig|1218.91.peg.2,peg,MIT9313_1368_2120,1368,2120,+,RNA metabolism-related protein,NaN,NaN,NaN,ttgagtgatttgttgcatcatcgtgtacgttgtgatcaggggttag...,MSDLLHHRVRCDQGLDHGPGVLPWMHPPVHRLLGWVSRPSALRVSR...,MIT9313,1368,2120
2,MIT9313,PMIT9313_0003,fig|1218.91.peg.3,peg,MIT9313_2160_4562,2160,4562,+,"Phosphoribosylformylglycinamidine synthase, sy...",NaN,FIG01303876,icw(1);De_Novo_Purine_Biosynthesis,gtgtgtcagggccccgatttgagagttgactatgacgtggctgcgg...,MCQGPDLRVDYDVAAALRHEGLKPHDYDEICRRLQRAPNRVELGMF...,MIT9313,2160,4562
3,MIT9313,PMIT9313_0004,fig|1218.91.peg.4,peg,MIT9313_4622_6079,4622,6079,+,Amidophosphoribosyltransferase (EC 2.4.2.14),NaN,FIG00000179,isu;De_Novo_Purine_Biosynthesis,atgtgtggcattgttggcattgtttctactgcgctggtcaatcagc...,MCGIVGIVSTALVNQQIYDSLLLLQHRGQDSTGIATMDGSVFHLHK...,MIT9313,4622,6079
4,MIT9313,PMIT9313_0005,fig|1218.91.peg.5,peg,MIT9313_8604_6115,8604,6115,-,DNA gyrase subunit A (EC 5.99.1.3),NaN,FIG00000080,idu(1);DNA_gyrase_subunits idu(1);DNA_topoisom...,atggccgaggagcgccttcaaccaatcgcattgcatcaagagatgc...,MAEERLQPIALHQEMQRSYLEYAMSVIVGRALPDARDGLKPVQRRI...,MIT9313,6115,8604


In [6]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO

In [7]:
# get all sequence records for the specified genbank file
recs = [rec for rec in SeqIO.parse(os.path.join('data', "MIT9313.gb"), "genbank")]

# print the number of sequence records that were extracted
len(recs)


1

In [8]:
recs[0].features[1].location, recs[0].features[2].qualifiers, recs[0].features[2].strand

(FeatureLocation(ExactPosition(173), ExactPosition(1340), strand=1),
 OrderedDict([('locus_tag', ['AKG35_RS00005']),
              ('old_locus_tag', ['PMT0001', 'RG24_RS00005']),
              ('EC_number', ['2.7.7.7']),
              ('inference',
               ['COORDINATES: similar to AA sequence:RefSeq:WP_011129380.1']),
              ('note',
               ['Derived by automated computational analysis using gene prediction method: Protein Homology.']),
              ('codon_start', ['1']),
              ('transl_table', ['11']),
              ('product', ['DNA polymerase III subunit beta']),
              ('protein_id', ['WP_011129380.1']),
              ('db_xref', ['GI:499441916']),
              ('translation',
               ['MKLVCSQAELNTALQLVSRAVASRPTHPVLANVLLTADAGTDRLSLTGFDLSLGIQTALTASVESSGAITLPARLLGEIVARLSSDSPITLATDEAGEQVELKSSSGSYQMRGMSADDFPELPLVESGKAVKVNARALLTALRGTLFASSSDEAKQLLTGVHLHFDGKAMEAAATDGHRLAVLSLADALAVETTVSSEMDDEGENFSVTLPSRSLREVERLIAGWRGDDQVSLFCDKGQVVFLAADQVVTSR

In [9]:
recs[0].features[1].location, recs[0].features[1].qualifiers, recs[0].features[2].strand

(FeatureLocation(ExactPosition(173), ExactPosition(1340), strand=1),
 OrderedDict([('locus_tag', ['AKG35_RS00005']),
              ('old_locus_tag', ['PMT0001', 'RG24_RS00005'])]),
 1)

In [10]:
recs[0].features[4].location, recs[0].features[4].qualifiers, recs[0].features[4].strand

(FeatureLocation(ExactPosition(1343), ExactPosition(2120), strand=1),
 OrderedDict([('locus_tag', ['AKG35_RS00010']),
              ('old_locus_tag', ['PMT0002', 'RG24_RS00010']),
              ('inference',
               ['COORDINATES: similar to AA sequence:RefSeq:WP_011824692.1']),
              ('note',
               ['Derived by automated computational analysis using gene prediction method: Protein Homology.']),
              ('codon_start', ['1']),
              ('transl_table', ['11']),
              ('product', ['hypothetical protein']),
              ('protein_id', ['WP_041384224.1']),
              ('db_xref', ['GI:752737102']),
              ('translation',
               ['MNLPDQILLSDLLHHRVRCDQGLDHGPGVLPWMHPPVHRLLGWVSRPSALRVSRDVWRLDQSRGINEQEVFVKGQPAVSDQITLERLPTLLDADLLDRDGHRLGLVADFVFIPTTGKILHYLVSRSDPRLPGSSRWRLTTDRIVDQQPGIVATALRGLDDLPLVHSSVRQGLIKRSRHWRDQLHKMGDRASDRLEGWLEDPPWDQPVERSWKSSDMAEMDPLESWDDEQTDNFSQSTEELRHRGRPDGRGDGQDDPWV'])]),
 1)

In [11]:
int(recs[0].features[1].location.end)

1340

In [12]:
genome_df.columns

Index(['contig_id', 'gene_id', 'feature_id', 'type', 'location', 'start',
       'stop', 'strand', 'function', 'aliases', 'figfam', 'evidence_codes',
       'nucleotide_sequence', 'aa_sequence', 'genome', 'left', 'right'],
      dtype='object')

In [13]:
{i.type for i in recs[0].features}

{'CDS', 'gene', 'ncRNA', 'rRNA', 'regulatory', 'source', 'tRNA', 'tmRNA'}

In [14]:
i = recs[0].features[2]

In [15]:
i.qualifiers

OrderedDict([('locus_tag', ['AKG35_RS00005']),
             ('old_locus_tag', ['PMT0001', 'RG24_RS00005']),
             ('EC_number', ['2.7.7.7']),
             ('inference',
              ['COORDINATES: similar to AA sequence:RefSeq:WP_011129380.1']),
             ('note',
              ['Derived by automated computational analysis using gene prediction method: Protein Homology.']),
             ('codon_start', ['1']),
             ('transl_table', ['11']),
             ('product', ['DNA polymerase III subunit beta']),
             ('protein_id', ['WP_011129380.1']),
             ('db_xref', ['GI:499441916']),
             ('translation',
              ['MKLVCSQAELNTALQLVSRAVASRPTHPVLANVLLTADAGTDRLSLTGFDLSLGIQTALTASVESSGAITLPARLLGEIVARLSSDSPITLATDEAGEQVELKSSSGSYQMRGMSADDFPELPLVESGKAVKVNARALLTALRGTLFASSSDEAKQLLTGVHLHFDGKAMEAAATDGHRLAVLSLADALAVETTVSSEMDDEGENFSVTLPSRSLREVERLIAGWRGDDQVSLFCDKGQVVFLAADQVVTSRTLDGTYPNYRQLIPDGFARSFDVDRRAFIAALERIAVLADQHNNVVKVSGDNTSELLQISADAQDVGSGSESLSAEFTGDAVQ

In [16]:
    merge_cols = ['contig_id', 'gene_id', 'location', 'type', 
                  'start', 'stop', 'strand',
                  'function', 'figfam',
                  'nucleotide_sequence', 'aa_sequence', 'genome', 'left', 'right']


In [17]:

def gb_to_record(i, genome='MIT9313'):
    q = i.qualifiers
    g = lambda x : ','.join(q.get(x, []))
    pmtid = [i for i in q.get('old_locus_tag',[]) if i.startswith('PMT')]
    record = {
        'contig_id' : genome, 
        'gene_id' : g('locus_tag'), 
        'pmt_id': ','.join(pmtid), 
        'type' : i.type, 
        'location' : g('locus_tag'),  
        'strand' : '+' if i.location.strand == 1 else '-',
        'start' : int(i.location.start),
        'stop' : int(i.location.end),
        'left' : int(i.location.start),
        'right' : int(i.location.end),
        'function' : g('product'), 
        'genome' : genome, 
        'old_locus_tag' : g('old_locus_tag'), 
        'product' : g('product'), 
        'db_xref' : g('db_xref'), 
        'protein_id' : g('protein_id'), 
        'figfam' : '',
        'nucleotide_sequence' : '', 
        'aa_sequence' : g('translation'), 
    }
    return record


In [18]:
types_to_collect = ['CDS', 'ncRNA', 'rRNA', 'regulatory', 'tRNA', 'tmRNA']
records = [gb_to_record(i) for i in recs[0].features if i.type in types_to_collect]

In [19]:
gdf = pd.DataFrame(records)

In [23]:
gdf.to_feather(os.path.join('data', f'{genome}.ncbi.gb.feather.gz'))


In [20]:
gdf.loc[gdf.type != 'CDS'].head(20)

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
44,MIT9313,AKG35_RS00225,,tRNA,AKG35_RS00225,-,42281,42353,42281,42353,tRNA-Gly,MIT9313,"RG24_RS00225,RNA_44",tRNA-Gly,,,,,
84,MIT9313,AKG35_RS00430,,tRNA,AKG35_RS00430,+,87819,87891,87819,87891,tRNA-Gly,MIT9313,"RG24_RS00430,RNA_1",tRNA-Gly,,,,,
162,MIT9313,,,regulatory,,+,178355,178459,178355,178459,,MIT9313,,,RFAM:RF00059,,,,
187,MIT9313,AKG35_RS00920,,rRNA,AKG35_RS00920,+,208846,210328,208846,210328,16S ribosomal RNA,MIT9313,"RG24_RS00925,RNA_53",16S ribosomal RNA,,,,,
188,MIT9313,AKG35_RS00925,,tRNA,AKG35_RS00925,+,210510,210584,210510,210584,tRNA-Ile,MIT9313,"RG24_RS00930,RNA_2",tRNA-Ile,,,,,
189,MIT9313,AKG35_RS00930,,tRNA,AKG35_RS00930,+,210593,210666,210593,210666,tRNA-Ala,MIT9313,"RG24_RS00935,RNA_3",tRNA-Ala,,,,,
190,MIT9313,AKG35_RS00940,,rRNA,AKG35_RS00940,+,211155,214034,211155,214034,23S ribosomal RNA,MIT9313,"RG24_RS00945,RNA_54",23S ribosomal RNA,,,,,
191,MIT9313,AKG35_RS00945,,rRNA,AKG35_RS00945,+,214141,214258,214141,214258,5S ribosomal RNA,MIT9313,"RG24_RS00950,RNA_55",5S ribosomal RNA,RFAM:RF00001,,,,
207,MIT9313,AKG35_RS01015,,tRNA,AKG35_RS01015,-,227744,227816,227744,227816,tRNA-Thr,MIT9313,"RG24_RS01020,RNA_43",tRNA-Thr,,,,,
208,MIT9313,AKG35_RS01020,,tRNA,AKG35_RS01020,-,227826,227908,227826,227908,tRNA-Tyr,MIT9313,"RG24_RS01025,RNA_42",tRNA-Tyr,,,,,


In [21]:
genome = 'MIT9313'
accession = sf.get_accession(genome)
accession

'BX548175'

In [22]:
orf_dpath = 'orffinder_tmp'
orf_fpath = os.path.join(orf_dpath, f'{accession}.orffinder.fa')
df = sf.parse_orffinder_fasta(orf_fpath)

In [80]:
df['rast_left'] = df['left']
df['rast_right'] = df['right'] + 1

In [81]:
gdf.loc[gdf.location == 'AKG35_RS00020']

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
3,MIT9313,AKG35_RS00020,PMT0004,CDS,AKG35_RS00020,+,4621,6079,4621,6079,amidophosphoribosyltransferase,MIT9313,"PMT0004,RG24_RS00020",amidophosphoribosyltransferase,GI:499441919,WP_011129383.1,,,MCGIVGIVSTALVNQQIYDSLLLLQHRGQDSTGIATMDGSVFHLHK...


In [89]:
df.loc[(df.rast_left > 4620) & (df.rast_left < 4630),  ].T

,5047
orfid,ORF5048_BX548175.1:4621:6078
contig,BX548175.1
left,4621
right,6078
start,4621
stop,6078
strand,+
aaseq,MCGIVGIVSTALVNQQIYDSLLLLQHRGQDSTGIATMDGSVFHLHK...
len_nn,1458
len_aa,485


In [83]:
df = sf.add_gene_annotation_to_sorf_df(df, gdf)
df = sf.add_overlapping_genes(df, gdf)

In [84]:
summary_df = sf.gen_summary_df(df, genome)
df.to_feather(os.path.join(orf_dpath, f'{genome}.ncbi.sorf.feather.gz'))
summary_df.to_csv(os.path.join(orf_dpath, f'{genome}.ncbi.sorf.summary.csv.gz'))


In [85]:
summary_df

,index,otype,count,subset,genome
0,0,antisense,13413,all,MIT9313
1,1,in_frame,1443,all,MIT9313
2,2,known,1472,all,MIT9313
3,3,out_frame,8841,all,MIT9313
4,4,overlap,1494,all,MIT9313
5,5,standalone,4667,all,MIT9313
6,0,antisense,12985,<100aa,MIT9313
7,1,in_frame,1209,<100aa,MIT9313
8,2,known,300,<100aa,MIT9313
9,3,out_frame,8504,<100aa,MIT9313
